In [583]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np

result_dir = os.path.join(os.getcwd(), "results")
results = os.listdir(result_dir)

def load(f):
    file_dir = os.path.join(result_dir, f, "results.json")
    with open(file_dir, "r") as f:

        results = json.load(f)
    return results

# create a function that captalizes the first letter of a word
def capitalize(string):
    return string[0].upper() + string[1:]

def load_from_query(query):
    results = []
    for f in os.listdir(result_dir):
        if f.find(query) != -1:
            return load(f)
    return None

def show(data, scale=1):
    for d in data:
        print("|".join([f"{x*scale if x else 0:.4f}" for x in d]))
        
cache_sizes = [5, 10, 30, 50, 80, 100]
NR = 3
Nr = 30

avg_delay = []
hit_ratio = []
success_ratio = []
labels = []

for cache in ["random","mcfed","avgfed","nocache",]:
    for delivery in ["random", "greedy", "drl", "norsu"]:
                if cache in ["random", "avgfed", "nocache"] and delivery not in [
                    "drl",
                    "norsu",
                ]:
                    continue
                if delivery == "norsu" and cache != "random":
                    continue
                
                labels.append(f"{capitalize(cache)}_{capitalize(delivery)}")
                sub_data1 = []
                sub_data2 = []
                sub_data3 = []
                for cache_size in cache_sizes:
                    query = f"{NR}_{cache_size}_{Nr}_{cache}_{delivery}"
                    result = load_from_query(query)
                    if result:
                        sub_data1.append(np.mean(result["round_avg_delay_tracking"]))
                        sub_data2.append(np.mean(result["round_hit_ratio_tracking"]))
                        sub_data3.append(np.mean(result["round_success_ratio_tracking"]))
                    else:
                        sub_data1.append(None)
                        sub_data2.append(None)
                        sub_data3.append(None)
                avg_delay.append(sub_data1)
                hit_ratio.append(sub_data2)
                success_ratio.append(sub_data3)

In [584]:
labels

['Random_Drl',
 'Random_Norsu',
 'Mcfed_Random',
 'Mcfed_Greedy',
 'Mcfed_Drl',
 'Avgfed_Drl',
 'Nocache_Drl']

In [585]:
show(avg_delay, 1000)

186.8738|186.8551|186.0597|185.8613|184.9967|184.6660
365.2666|364.3428|364.2805|362.8977|364.9506|364.1090
225.2321|224.0670|225.0280|219.3653|226.3719|218.8423
205.8209|190.4602|196.0688|177.1901|176.6276|162.3408
185.7054|184.1283|181.5899|179.2223|177.1178|172.2337
185.3326|184.4758|181.1782|178.5757|174.4519|175.3588
186.9231|187.0024|187.0435|186.9239|187.1567|186.9664


In [586]:
show(hit_ratio)

0.0014|0.0036|0.0134|0.0194|0.0363|0.0453
0.0000|0.0000|0.0000|0.0000|0.0000|0.0000
0.0189|0.0304|0.0627|0.0941|0.1328|0.1584
0.0672|0.1116|0.2339|0.3198|0.4875|0.5865
0.0226|0.0409|0.0809|0.1246|0.1560|0.2324
0.0246|0.0386|0.0926|0.1334|0.1936|0.1873
0.0000|0.0000|0.0000|0.0000|0.0000|0.0000
